In [1]:
#dataset import

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
#Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

#用来验证和计算准确率的样本数，如果内存不够，可以调小这个数字
test_valid_size = 256

#神经网络参数
n_classes = 10
dropout = 0.75

### Weights and Biases

In [3]:
#Store layers weight & bias
weights = {
    #H W D = 5 5 1(黑白所以为1，有颜色则为3),输出深度为32
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    #前一个输出为下一个深度
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

### 卷积 

In [4]:
def conv2d(x, w, b, strides=1):
    x = tf.nn.conv2d(x, w, strides=[1, strides, strides, 1], padding='SAME')
    X = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

### 最大池化

tf.nn.max_pool() 通过设定ksize参数来设定滤波器大小，从而实现最大池化

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize = [1, k, k, 1],
        strides = [1, k, k, 1],
        padding = 'SAME')

### 模型

In [11]:
def conv_net(x, weights, biases, dropout):
    #layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    #layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    #Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

### Session 

In [13]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

#Model
logits = conv_net(x, weights, biases, keep_prob)

#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initializing the variables
init = tf.global_variables_initializer()

#launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                    x: batch_x,
                    y: batch_y,
                    keep_prob: dropout})
            
            #Caculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                    x: batch_x,
                    y: batch_y,
                    keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                    x: mnist.validation.images[:test_valid_size],
                    y: mnist.validation.labels[:test_valid_size],
                    keep_prob:1.})
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:>.6f}'.format(
                    epoch + 1,
                    batch + 1,
                    loss, 
                    valid_acc))
            
            #Calculate Test Accuracy
            test_acc = sess.run(accuracy, feed_dict = {
                    x: mnist.test.images[:test_valid_size],
                    y: mnist.test.labels[:test_valid_size],
                    keep_prob: 1.})
            print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 49791.0781 Validation Accuracy: 0.125000
Testing Accuracy: 0.09765625
Epoch  1, Batch   2 -Loss: 43789.5195 Validation Accuracy: 0.121094
Testing Accuracy: 0.08203125
Epoch  1, Batch   3 -Loss: 41745.8086 Validation Accuracy: 0.109375
Testing Accuracy: 0.09765625
Epoch  1, Batch   4 -Loss: 39418.7969 Validation Accuracy: 0.125000
Testing Accuracy: 0.1015625
Epoch  1, Batch   5 -Loss: 35869.7969 Validation Accuracy: 0.117188
Testing Accuracy: 0.11328125
Epoch  1, Batch   6 -Loss: 33067.2500 Validation Accuracy: 0.128906
Testing Accuracy: 0.12109375
Epoch  1, Batch   7 -Loss: 27862.1602 Validation Accuracy: 0.136719
Testing Accuracy: 0.12109375
Epoch  1, Batch   8 -Loss: 25986.1113 Validation Accuracy: 0.132812
Testing Accuracy: 0.1328125
Epoch  1, Batch   9 -Loss: 26047.1113 Validation Accuracy: 0.140625
Testing Accuracy: 0.125
Epoch  1, Batch  10 -Loss: 27453.4180 Validation Accuracy: 0.148438
Testing Accuracy: 0.12109375
Epoch  1, Batch  11 -Loss: 26312.1309

### TensorFlow卷积训练 

In [14]:
import tensorflow as tf
import numpy as np

x = np.array([
        [0, 1, 0.5, 10],
        [2, 2.5, 1, -8],
        [4, 0, 5, 6],
        [15, 1, 2, 3]], dtype = np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def conv2d(input):
    F_W = tf.Variable(tf.truncated_normal((2, 2, 1, 3)))
    F_b = tf.Variable(tf.zeros((3)))
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

out = conv2d(X)

out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))

out_width  = ceil(float(in_width - filter_width + 1) / float(strides[2]))

### TensorFlow池化 

In [15]:
import tensorflow as tf
import numpy as np

x = np.array([
        [0, 1, 0.5, 10],
        [2, 2.5, 1, -8],
        [4, 0, 5, 6],
        [15, 1, 2, 3]], dtype = np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    ksize = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.max_pool(input, ksize, strides, padding)

out = maxpool(X)